In [1]:
import nest_asyncio
from dotenv import load_dotenv

# This is required only for jupyter notebooks, since they have their own eventloop
nest_asyncio.apply()
load_dotenv()  # This will load the variables from the .env file into the environment

True

In [2]:
from langchain_modules.toolkit import PlayWrightBrowserToolkit
from langchain_modules.tools.playwright.utils import create_async_playwright_browser

async_browser = create_async_playwright_browser(headless=False)
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)

tools = toolkit.get_tools()

name_to_tool_map = {tool.name: tool for tool in tools}

In [4]:
navigate = name_to_tool_map["navigate_browser"]

await navigate._arun(**{"url": "https://dev.app.serenityapp.com/sign-in"})

<Response url='https://dev.app.serenityapp.com/sign-in' request=<Request url='https://dev.app.serenityapp.com/sign-in' method='GET'>>


'Navigating to https://dev.app.serenityapp.com/sign-in returned status code 200'

In [60]:
fill_element = name_to_tool_map["fill_element"]

await fill_element._arun(
    **{"selector": "input[name='email']", "text": "viktor.nawrath+3s1@profiq.com"}
)
await fill_element._arun(**{"selector": "input[name='password']", "text": "Serenity!"})

click_element = name_to_tool_map["click_element"]

await click_element._arun(**{"selector": "button[data-testid='SignInPage-submit']"})

"Unable to click on element 'button[data-testid='SignInPage-submit']'"

In [64]:
async def is_element_in_viewport(element, viewport_width, viewport_height):
    bounding_box = await element.bounding_box()
    if not bounding_box:
        return False
    x, y, width, height = (
        bounding_box["x"],
        bounding_box["y"],
        bounding_box["width"],
        bounding_box["height"],
    )
    return (
        x >= 0
        and y >= 0
        and x + width <= viewport_width
        and y + height <= viewport_height
    )


async def amark_invisible_elements(page):
    # Get viewport size
    viewport_size = page.viewport_size
    viewport_width = viewport_size["width"]
    viewport_height = viewport_size["height"]

    body = await page.query_selector("body")
    for element in await body.query_selector_all("*"):
        if not await is_element_in_viewport(element, viewport_width, viewport_height):
            await element.evaluate('el => el.setAttribute("data-visible", "false")')

In [68]:
from bs4 import BeautifulSoup, Tag


def clean_attributes(tag):
    # List of attributes to remove, add or remove attributes as needed
    blocked_attrs = ["class", "style"]
    tag.attrs = {k: v for k, v in tag.attrs.items() if k not in blocked_attrs}


def remove_specific_tags(tag):
    # List of tags to remove
    tags_to_remove = ["path", "meta", "link", "noscript", "script", "style", "title"]
    for t in tag.find_all(tags_to_remove):
        t.decompose()


def remove_elements_by_data_attribute(tag, attribute, value):
    for elem in tag.select(f'[{attribute}="{value}"]'):
        elem.decompose()


def strip_html_recursively(tag):
    clean_attributes(tag)
    for child in tag.find_all(
        True, recursive=False
    ):  # `recursive=False` to only go one level deep
        if isinstance(child, Tag):
            strip_html_recursively(child)


def strip_html_to_structure(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    remove_specific_tags(soup)  # Remove specific tags before processing
    remove_elements_by_data_attribute(soup, "data-visible", "false")
    strip_html_recursively(soup)
    return str(soup)

In [69]:
from langchain.tools.playwright.utils import aget_current_page

page = await aget_current_page(toolkit.async_browser)

await amark_invisible_elements(page)

html_content = await page.content()

print(strip_html_to_structure(html_content))

<!DOCTYPE html>
<html lang="en"><head></head><body><div id="root"><div><header data-testid="AppLayoutAppBar"><div><div><div><div><svg viewbox="0 0 135 34"></svg></div></div><button data-testid="AppBar-switch-role-button" tabindex="0" type="button">Messaging<span><svg aria-hidden="true" data-testid="KeyboardArrowDownIcon" focusable="false" viewbox="0 0 24 24"></svg></span><span></span></button></div><div><div data-testid="AppBar-my-account-button" role="button" tabindex="0"><div><svg aria-hidden="true" data-testid="ArrowDropDownIcon" focusable="false" viewbox="0 0 24 24"></svg></div><div><span>Viktor Staff 1</span><p>Redux Test</p></div><div><div color="primary" data-testid="Avatar"><img alt="VS" src="https://d26d9t3siorj89.cloudfront.net/protected/us-east-2:c42e49dd-2414-4af4-8cf7-eae0497595eb/6dec5b36-09b6-45ed-bdd0-9a10487d7529.jpeg"/></div></div><span></span></div></div></div></header><div><div data-testid="View"><div></div></div><div><div><div><div></div></div></div></div></div><ma